<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/AssiyaKaratay_Assignment_4_Wiki_Categories_SparkRDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Info 
Assignment 4
MET CS777 Big Data Analytics

Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028

#### import libraries

In [3]:
!pip install --ignore-installed -q pyspark==3.1.2 

     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 198 kB 58.8 MB/s 


In [43]:
import os
import sys
import requests
import numpy as np
from operator import add

from pyspark import SparkContext
from pyspark.sql import SparkSession
import re
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

#### set up the Google Drive

In [5]:
#### set up the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# choose where project files will be saved
project_folder = "/content/drive/MyDrive/CS777_BigDataAnalytics/Assignment4/"
# project_folder = sys.argv[2]
# change the OS to use the project folder as the working directory
os.chdir(project_folder)

print('\n Working directory was changed to ' + project_folder )


 Working directory was changed to /content/drive/MyDrive/CS777_BigDataAnalytics/Assignment4/


### Task 1 Generate a 20K dictionary (10 points)

#### 1.1 The top 20,000 English words
Using Wikipedia pages, find the top 20,000 English words, save them in an array, and sort them based on the frequency of the occurrence.

In [7]:
def parsing(lines):
  # divide the data defore and after url
  division = lines.split('" url')
  # strip all characters except DOC ID
  id = division[0].split('<doc id="')[1] 
  # strip the end of title and last 6 characters containing '.</doc' .
  text = division[1].split('">')[1][:-6]
  # check the UNICODE decoding for readability 
  # and lowercase to count words with the same letters together
  regex = re.compile(r'[^a-zA-Z]', re.UNICODE).split(text.lower())
  return (id,regex)

In [8]:
# 1. get the file
wikiPagesFile= project_folder + "WikipediaPagesOneDocPerLine1000LinesSmall.txt.bz2"
# 2. create an rdd of data
wikiPages = sc.textFile(wikiPagesFile)
# 3. check if each line has id and url
validLines = wikiPages.filter(lambda x : 'id' in x and 'url=' in x)
# 4. trim unnecessary symbols and get (docID, listOfWords) pairs
listOfWords = validLines.map(parsing)
# 5. make every word a tuple dropping DocID
wordsAsTuples = listOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
# 6. count number of words in a corpus
counts=wordsAsTuples.reduceByKey(lambda x, y: x+y)
# 7. get the top 20K in a corpus
n = 20000
topWords = counts.top(n, lambda x: x[1])
# 8. create an empty RDD that has the number 0 through 20000
emptyRDD = sc.parallelize(range(n))
# 9. order words in descending popularity order, where  
# the word in position 0 is the most common used word
dictionary = emptyRDD.map (lambda x : (topWords[x][0], x))
# 10. save the output 
dictionary.saveAsTextFile(project_folder+'task11_topEnglishWords')
print("The top 20,000 English words", dictionary.top(5, lambda x : -x[1]))

The top 20,000 English words [('', 0), ('the', 1), ('of', 2), ('and', 3), ('in', 4)]


#### 1.2 docID as key and a Numpy array for the position of each word
As a result, a dictionary has been generated that contains the top 20K most frequent words in the corpus. Next go over each Wikipedia document and check if the words appear in the Top 20K words. At the end, produce an RDD that includes
the docID as key and a Numpy array for the position of each word in the top 20K dictionary.
(docID, [dictionaryPos1, dictionaryPos2, dictionaryPos3...])

In [9]:
def numberOfOccurrences(docIDAndPos):
    docID = docIDAndPos[0]
    listOfIndices = docIDAndPos[1]
    # create an array of zeros
    returnVal = np.zeros(20000)
    # count the occurrence of words e.g. there are 514 'my' in docID1, where 
    # 'my' is in the position 0 in the corpus
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    return (docID, returnVal)

In [10]:
# 1. get ("word1", docID) pairs from (docID, ["word1", "word2", "word3", ...])
allWordsWithDocID = listOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
# 2. inner join with dictionary to get a set of ("word1", (dictionaryPos, docID)) pairs
allDictionaryWords = dictionary.join(allWordsWithDocID)
# 3. drop the word as string to get a set of (docID, dictionaryPos) pairs
docIDAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))
# 4. get a pair (docID, np.array('count of word in pos1', 'count of word in pos2',...))
allDictionaryWordsInDocID = docIDAndPos.groupByKey()
IDPosArray = allDictionaryWordsInDocID.map(numberOfOccurrences)
# 5. save the output
IDPosArray.saveAsTextFile(project_folder+'task12_wordOccurrences')

### Task 2 - Create the TF-IDF Array (20 Points)
#### TF
After having the top 20K words we want to create a large array that its
columns are the words of the dictionary with number of occurrences of each word and the rows are documents.
The first step is calculating the “Term Frequency”, TF (x, w), vector for each document as follows:
“Term Frequency” is an indication of the number of times a term occurs in a document.
Numerator is number of occurrences of a word, and the denominator is the sum of all the words of the document.

In [11]:
def termFrequency(IDPosArray):
    numberOfWords = np.sum(IDPosArray[1])
    returnVal = np.divide(IDPosArray[1], numberOfWords)
    return (IDPosArray[0], returnVal)
# Now get (docID, [TF1, TF2, ...]) pairs
tf = IDPosArray.map(termFrequency)

#### IDF
Next, calculate “Inverse Document Frequency” for all the documents and finally
calculate TF-IDF(w) and create TF-IDF matrix of the corpus:
Note that the “size of corpus” is total number of documents (numerator).
To learn more about TF-IDF see the Wikipedia page:
https://en.wikipedia.org/wiki/Tf-idf

In [12]:
def occurrenceSign(tf):
    # empty np array of size 20K
    returnVal = np.zeros (20000)
    # get positions in dictionary of which words occurred in a doc
    tfZero = np.where(tf[1]>0)[0]
    for index in tfZero:
        if returnVal[index] == 0: 
          returnVal[index] = 1
    # A zero means that the word does not occur,one means that it does.
    return (tf[0], returnVal)

In [42]:
# 1. build occurrence table of words as 1 and 0
zeroOrOne = tf.map(occurrenceSign)
# 2. count the number of documents where each word in a dict appeared
dfArray = zeroOrOne.values().sum()
# 3. create an array of numberOfDocs of size 20K
numberOfDocs = wikiPages.count()
multiplier = np.full(n, numberOfDocs)
# 4. get the version of dfArray where the i^th entry is the idf for the i^th word in the corpus
idfArray = np.log(np.divide(multiplier, dfArray))
# 5. convert all of the tf vectors to tf*idf vectors
tfidf = tf.map(lambda x: (x[0], np.multiply(x[1], idfArray)))
# 6. save the output
while True:
      try:  
        tfidf.saveAsTextFile(project_folder+'task2_tfidf')        
        break
      except:
        break


### Task 3 - Implement the getPrediction function (30 Points)
Finally, implement the function getPrediction(textInput, k), which will predict the
membership of the textInput to the top 20 closest documents, and the list of top
categories.
You should use the cosine similarity to calculate the distances.

In [15]:
def parsingCategories(lines):
  # separate the data before and after comma
  division = lines.replace('"', '').split(',')
  return (division[0],division[1])

In [40]:
# a function that returns the prediction for the label of a string, using a kNN algorithm
def getPrediction (textInput, k):
    # 1. create an rdd of the text input
    myDoc = sc.parallelize (('', textInput))
    # 2. get (word, 1) pairs and preprocess the data
    textWords = myDoc.flatMap (lambda x : ((j, 1) for j in re.compile(r'[^a-zA-Z]', re.UNICODE).sub(' ', x).lower().split()))
    # 3. join dict(word, pos) with text words(word,1) to get (word, (dictionaryPos, 1)) pairs
    myDocAndDictJoined = dictionary.join (textWords)
    # 4. drop the string word and have (1,dictionaryPos)
    allDictionaryWordsInThatDoc = myDocAndDictJoined.map (lambda x: (x[1][1], x[1][0])).groupByKey()  
    # 5. array of occurrences (1, np.array(0, 2, 5, ...))
    IDPosArray = allDictionaryWordsInThatDoc.map(numberOfOccurrences)
    # 6. Get tf array for the text input 
    tf = IDPosArray.map(termFrequency).collect()[0][1]
    # 7. get tfidf array 
    myArrayTfidf = np.multiply (tf, idfArray)
    # 8. measure distance from the text tfidf to all cats tfidf, using cosine similarity (np.dot() )
    distances = featuresRDD.map (lambda x : (x[0], np.dot (x[1], myArrayTfidf)))
    # 9. get the top k largest distances (cos0 = 1 means similarity, cos90 = 0 means difference)
    topK = distances.top (k, lambda x : x[1])
    # 10. transform the top k distances into (cat_name, 1) pairs
    docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))
    # 11. for each cat_name, count the number of times this docID appeared in the top k
    numTimes = docIDRepresented.reduceByKey(add)
    # 12. Return the most occurred cat_names in the closest cat_names 
    topNumTimes = numTimes.top(k, lambda x: x[1])
    while True:
      try:  
        sc.parallelize(topNumTimes).saveAsTextFile(project_folder + 'task3_topCategories'+str(textInput[:6]))
        break
      except:
        break
    return topNumTimes

In [16]:
# 1. get categories file
wikiCategoryFile=project_folder + "wiki-categorylinks-small.csv.bz2"
# wikiCategoryFile = sys.argv[2]
# 2. create an rdd
wikiCategoryLinks=sc.textFile(wikiCategoryFile)
# 3. delete " and get (docID,cat_name) pairs
wikiCats = wikiCategoryLinks.map(parsingCategories)
# 4. join tfidf (docID, tdidf array) with categories (docID,cat_name)
catsTFIDFJoined = wikiCats.join(tfidf)
# 5. drop docID and obtain category name and tfidf array
featuresRDD = catsTFIDFJoined.map(lambda x: (x[1][0], x[1][1]))
# 6. Cache because kNN will be run on this data 
featuresRDD.cache()

('All_articles_with_unsourced_statements', array([0.        , 0.00404342, 0.00183544, ..., 0.        , 0.        ,
       0.        ]))


In [41]:
textInput = "Big data refers to data sets that are too large or complex to be \
dealt with by traditional data-processing application software. Data with many \
fields (rows) offer greater statistical power, while data with higher complexity\
 (more attributes or columns) may lead to a higher false discovery rate.[2] \
 Big data analysis challenges include capturing data, data storage, data \
 analysis, search, sharing, transfer, visualization, querying, updating, \
 information privacy, and data source. Big data was originally associated with \
 three key concepts: volume, variety, and velocity.[3] The analysis of big data \
 presents challenges in sampling, and thus previously allowing for only \
 observations and sampling. Thus a fourth concept, veracity, refers to the \
 quality or insightfulness of the data. Without sufficient investment in \
 expertise for big data veracity, then the volume and variety of data can \
 produce costs and risks that exceed an organization's capacity to create \
 and capture value from big data.[4]"

getPrediction(textInput, 20)


[('All_stub_articles', 3),
 ('Programming_language_topic_stubs', 2),
 ('Data_modeling_languages', 2),
 ('XML-based_standards', 2),
 ('All_Wikipedia_articles_needing_context', 1),
 ('All_articles_lacking_sources', 1),
 ('All_pages_needing_cleanup', 1),
 ('Articles_with_multiple_maintenance_issues', 1),
 ('Computer_networks', 1),
 ('Computing_stubs', 1),
 ('Use_dmy_dates_from_May_2019', 1),
 ('All_articles_needing_additional_references', 1),
 ('Articles_needing_additional_references_from_May_2016', 1),
 ('Articles_lacking_sources_from_December_2009', 1),
 ('Wikipedia_articles_needing_context_from_October_2009', 1)]

### Task 4 – Implement the code using Dataframes (30 points)
Implement the complete code in Dataframe and print out the results of the task 3
using dataframes in pyspark. From the beginning of your code to the end of your
kNN implementation you are allowed to use spark dataframe and python (including
python libraries like numpy). You are not allowed to use RDDs.

### Task 5 - Removing Stop Words and Do Stemming (10 points)
Task 5.1 - Remove Stop Words (5 point)
Describe if removing the English Stop words (most common words like ”a,
the, is, are, i, you, ...”) would change the final kNN results.
Does your result change significantly after removing the stop words? Why?
Provide reasons.
You do not need to implement this task.
Task 5.2 – Considering English word stemming (5 point)
We can stem the words [”game”,”gaming”,”gamed”,”games”] to their root
word ”game”.
Does stemming change your result significantly? Why? Provide reasons.
You can learn more about stemming at:
https://en.wikipedia.org/wiki/Stemming
You do not need to implement this task.